# After refactoring...

In [2]:
import signalflow as sf
from pixasonics.core import App, Mapper
from pixasonics.features import MeanPixelValue
from pixasonics.synths import Theremin

img_path = "images/cellular_dataset/Timepoint_001_220518-ST_C03_s1.jpg"

# Create app
app = App()
app.load_image(img_path) # Load image

# Create objects
mean_pix = MeanPixelValue()
theremin = Theremin()
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_high=1000)

# Add objects to app
app.add_feature(mean_pix)
app.add_synth(theremin)
app.add_mapper(pix2freq)


MultiCanvas(height=580, width=1080)

IntSlider(value=50, description='Probe Width', layout=Layout(width='400px'), max=500, min=1, style=SliderStyle…

IntSlider(value=50, description='Probe Height', layout=Layout(width='400px'), max=500, min=1, style=SliderStyl…

ToggleButton(value=False, description='Audio', layout=Layout(width='auto'), style=ToggleButtonStyle(text_color…

FloatSlider(value=0.0, description='Master Volume (dB)', layout=Layout(width='400px'), max=0.0, min=-36.0, ste…

...

Text(value='', description='MeanPixelValue_0_Min:', disabled=True, style=TextStyle(description_width='auto'))

Text(value='', description='MeanPixelValue_0_Max:', disabled=True, style=TextStyle(description_width='auto'))

Text(value='', description='MeanPixelValue_0_Last:', disabled=True, style=TextStyle(description_width='auto'))

Button(description='Reset MeanPixelValue MinMax', layout=Layout(height='40px', width='auto'), style=ButtonStyl…

Label(value='Mapper: MeanPixelValue ==> Theremin Frequency')

In [11]:
# test adding another mapping: mean pixel value to amplitude with an exponential curve
pix2amp = Mapper(mean_pix, theremin["amplitude"], exponent=2)
app.add_mapper(pix2amp)

Label(value='Mapper: MeanPixelValue ==> Theremin Amplitude')

In [12]:
pix2pan = Mapper(mean_pix, theremin["panning"])
app.add_mapper(pix2pan)

Label(value='Mapper: MeanPixelValue ==> Theremin Panning')

In [13]:
app.mappers

[Mapper: MeanPixelValue ==> Theremin Frequency,
 Mapper: MeanPixelValue ==> Theremin Amplitude,
 Mapper: MeanPixelValue ==> Theremin Panning]

In [14]:
# remove a mapper
app.remove_mapper(pix2amp)
app.mappers

[Mapper: MeanPixelValue ==> Theremin Frequency,
 Mapper: MeanPixelValue ==> Theremin Panning]

In [15]:
# remove old frequency mapping and add a new one with different scaling
app.remove_mapper(pix2freq)
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_low=100, out_high=2000)
app.add_mapper(pix2freq)
app.mappers

Label(value='Mapper: MeanPixelValue ==> Theremin Frequency')

[Mapper: MeanPixelValue ==> Theremin Panning,
 Mapper: MeanPixelValue ==> Theremin Frequency]

In [16]:
graph = sf.AudioGraph.get_shared_graph()
print(graph.structure)
print(graph.status)

 * audioout
   input0:
    * multiply
      input0:
       * bus
         input0:
          * stereo-panner
            input:
             * multiply
               input0:
                * sine
                  frequency:
                   * smooth
                     input:
                      * buffer-player
                        loop: 1.000000
                        rate: 1.000000
                     smooth: 0.999425
               input1:
                * smooth
                  input:
                   * buffer-player
                     loop: 1.000000
                     rate: 1.000000
                  smooth: 0.999425
            pan:
             * smooth
               input:
                * buffer-player
                  loop: 1.000000
                  rate: 1.000000
               smooth: 0.999425
      input1:
       * smooth
         input:
          * multiply
            input0:
             * decibels-to-amplitude
               input: 0.000000
   